In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
PATH_pred = "/Users/theomathieu/Downloads/fullD3_pred/res/dist_pred.csv"
#PATH_pred = "/Users/theomathieu/Downloads/cadotte_15/dist_pred.csv"
#PATH_manual = "/Users/theomathieu/Downloads/exp/dist.csv"

In [ ]:
df_pred = pd.read_csv(PATH_pred)
df_pred["which"]= "pred"
#df_manual = pd.read_csv(PATH_manual)
#df_manual["which"]= "manual"

In [ ]:
display(df_pred.head())
#display(df_manual.head())
#concatenated_df = pd.concat([df_pred, df_manual], ignore_index=True)
concatenated_df = df_pred
z_scores = (concatenated_df['height'] - concatenated_df['height'].mean()) / concatenated_df['height'].std()
concatenated_df["z_scores"] = z_scores
fig = px.scatter(concatenated_df, x="level", y="z_scores", color="which", hover_data=['sub_name', 'height'])
fig.show()

In [ ]:
display(concatenated_df)

In [ ]:
fig = px.violin(df_pred, y="height", x="level", color="which", box=True, points="all",
          hover_data=["sub_name","which"])

fig.update_layout(title='Violin Plot for caudal-rostral length distribution',
                  xaxis=dict(title='Spinal level'),
                  yaxis=dict(title='caudal to rostral length (mm)'))
fig.show()

In [ ]:
fig = px.violin(df_pred, y="height", x="level", color="which", box=True, points="all",
          hover_data=["sub_name","which"])
fig.show()

In [ ]:
clean = concatenated_df.copy()
to_clean = []
#to_clean.append((clean["sub_name"] == "sub-amu02_215_0000") & (clean["wich"] == "manual"))
#to_clean.append((clean["sub_name"] == "sub-barcelona06_214_0000") & (clean["wich"] == "pred"))
to_clean.append((clean["z_scores"] > 2))
to_clean.append((clean["z_scores"] < -2))
deleted = pd.DataFrame()
for need in to_clean:
    deleted = pd.concat([deleted, clean[need]], ignore_index=True)
    clean = clean[~need]

comp =df_pred
for lvl in range(2, 12):
    #mean_c = np.mean(comp["height"][comp["level"]==lvl])
    #std_c = np.std(comp["height"][comp["level"]==lvl])
    mean_o = np.mean(concatenated_df["height"][concatenated_df["level"]==lvl])
    std_o = np.std(concatenated_df["height"][concatenated_df["level"]==lvl])
    #print(f"CLEAN|ORIG Height level {lvl} (mm): {mean_c:.2f} +/- {std_c:.2f}\tvs\t{mean_o:.2f} +/- {std_o:.2f}")
    print(f"Spinal level {lvl} length (mm): \t{mean_o:.2f} +/- {std_o:.2f}")

In [ ]:
deleted

In [ ]:
fig = px.violin(clean, y="height", x="level", color="which", box=True, points="all",
          hover_data=["sub_name","which"])
fig.show()

In [ ]:
fig = px.scatter(clean, x="level", y="z_scores", color="which", hover_data=['sub_name', 'height'])
fig.show()

In [ ]:
levels = ['C3', 'C4', 'C5', 'C6', 'C7', 'C8']
level_data = {
    'C3': {'mean': 10.5, 'std': 2.2},
    'C4': {'mean': 9.9, 'std': 1.3},
    'C5': {'mean': 10.5, 'std': 1.5},
    'C6': {'mean': 9.7, 'std': 1.6},
    'C7': {'mean': 9.4, 'std': 1.4},
    'C8': {'mean': 9.6, 'std': 1.4},
}

# Create a DataFrame to hold the data
data = clean.copy()

# Generate 20 subjects for each level
for i, level in enumerate(levels):
    mean = level_data[level]['mean']
    std = level_data[level]['std']
    subject_data = np.random.normal(loc=mean, scale=std, size=31)
    data = pd.concat([data, pd.DataFrame({'level': i+3, 'height': subject_data, 'spinal_start':0, 'spinal_end':1, 'vertebrae_start':0, 'vertebrae_end':1, 'which': 'generated', 'sub_name': 'generated_XX', 'z_scores':0})], ignore_index=True)

display(data)
# Create the violin plot with plotly.express
fig = px.violin(data, y="height", x="level", color="which", box=True, points="all",
          hover_data=["sub_name","which"])

fig.update_layout(title='Violin Plot for caudal-rostral length distribution',
                  xaxis=dict(title='Spinal level'),
                  yaxis=dict(title='caudal to rostral length (mm)'))
fig.show()